In [131]:
!pip install python-constraint

# Introdução

O presente projeto tem como objetivo a criação de uma inteligência artificial capaz de gerar, de forma automática e otimizada, um horário escolar. Pretende-se desenvolver uma solução eficiente que responda às diversas restrições e necessidades típicas da elaboração de horários em instituições de ensino, como distribuição das aulas, disponibilidade de salas e professores, entre outros critérios.

## Elementos da equipa

Diogo Marques - 27972
David Carvalho - 27973
Gonçalo Vidal - 27984
Gabriel Fortes - 27976
Diogo Caldas - 27951

# Variáveis

As variáveis correspondem às sessões (aulas) de cada unidade curricular (UC) para cada turma.
Cada turma (t01, t02, t03) tem um conjunto de unidades curriculares associadas:


In [132]:
classes = {
    "t01": ["UC11", "UC12", "UC13", "UC14", "UC15"],
    "t02": ["UC21", "UC22", "UC23", "UC24", "UC25"],
    "t03": ["UC31", "UC32", "UC33", "UC34", "UC35"]
}
classes_por_dia = {turma: {d: [] for d in range(5)} for turma in classes.keys()}

# Domínio

## Domínios dos Professores

Os domínios associados aos professores definem em que blocos horários (slots) cada docente pode lecionar, garantindo que o agente respeita as suas disponibilidades durante a geração dos horários.

### Atribuição de Unidades Curriculares

Cada professor tem associadas as unidades curriculares (UC) que leciona, permitindo identificar as aulas que pertencem ao seu horário.

In [133]:
teacher_uc = {
    "jo": ["UC11", "UC21", "UC22", "UC31"],
    "mike": ["UC12", "UC23", "UC32"],
    "rob": ["UC13", "UC14", "UC24", "UC33"],
    "sue": ["UC15", "UC25", "UC34", "UC35"],
}

### Representação dos Slots Horários

Cada professor tem 20 slots (de 0 a 19), correspondentes a 5 dias por semana com 4 blocos de aulas por dia.
O dicionário seguinte representa o horário-base vazio de cada professor.

In [134]:
teacher_schedule = {
    "jo": [0] * 20,
    "mike": [0] * 20,
    "rob": [0] * 20,
    "sue": [0] * 20,
}

### Restrições de Disponibilidade

As restrições de disponibilidade definem os slots em que cada professor está indisponível para lecionar.
Estas restrições são aplicadas como hard constraints, limitando os domínios das variáveis dos professores apenas aos horários possíveis.

In [135]:
teacher_restrictions = {
    "mike": [12, 13, 14, 15, 16, 17, 18, 19],
    "rob": [0, 1, 2, 3],
    "sue": [8, 9, 10, 11, 16, 17, 18, 19],
    "jo": []
}

## Domínio das Aulas Online

Algumas unidades curriculares podem ser lecionadas exclusivamente em formato online, o que significa que não necessitam de uma sala física associada.
Esta informação é representada no dicionário online_schedule, que indica quais as UCs com essa característica:

In [136]:
online_schedule = {
    "UC21": [2],
    "UC31": [2],
}

## Domínio das Salas

As salas fazem parte integrante das variáveis do problema, representando os espaços físicos onde as aulas presenciais decorrem.
Cada sessão de uma unidade curricular (UC) deve ser atribuída a uma sala disponível, respeitando os requisitos específicos de algumas UCs.

### Aulas com Sala Obrigatória

Determinadas unidades curriculares necessitam de uma sala específica (como um laboratório), onde todas as suas aulas devem ocorrer.
Estas salas obrigatórias estão definidas no dicionário room_schedule

In [137]:
room_schedule = {
    "lab01": ["UC14", "UC22"],
}

### Salas Disponíveis

As restantes unidades curriculares, sem exigência de sala específica, podem ser atribuídas a qualquer uma das salas listadas em available_rooms:

In [138]:
available_rooms = ["r01", "r02", "r03", "r04", "r05"] + list(room_schedule.keys())

## Geração dos Domínios de Disponibilidade dos Professores

Esta parte do código é responsável por gerar automaticamente os domínios de disponibilidade horária dos professores.

Primeiro, são definidos 20 slots correspondentes aos blocos horários da semana (cinco dias com quatro blocos por dia).

Em seguida, para cada professor, são identificados os slots em que está indisponível, de acordo com o dicionário teacher_restrictions.

Os slots restantes são considerados disponíveis e armazenados no dicionário teacher_available_slots, que passa a representar o domínio de horários possíveis para cada docente.

Além disso, é criado o dicionário teacher_by_uc, que associa cada unidade curricular ao professor responsável, permitindo que o agente aplique corretamente as restrições relacionadas com disponibilidade e sobreposição de horários.

Com esta informação, o sistema assegura que cada professor apenas é alocado a horários compatíveis com a sua disponibilidade real.

In [139]:
slots = list(range(20))
teacher_available_slots = {}

for teacher in teacher_schedule.keys():
    unavailable = set(teacher_restrictions.get(teacher, []))
    available = [s for s in slots if
                 s not in unavailable]  
    teacher_available_slots[teacher] = available  

teacher_by_uc = {uc: teacher for teacher, ucs in teacher_uc.items() for uc in ucs}

# Hard Constraints para Horário Escolar

Restrições Rígidas (Hard Constraints)

- Cada aula tem duração fixa de 2 horas.
- Cada curso tem uma ou duas aulas por semana.
- Nenhuma turma pode ter mais de 3 aulas por dia.
- O horário de cada aula deve respeitar a disponibilidade do professor.
- Não podem existir conflitos:
- Um professor não pode lecionar duas aulas ao mesmo tempo.
- Uma turma não pode assistir a duas aulas no mesmo horário.
- Uma sala não pode ser utilizada por mais de uma aula no mesmo bloco horário.
- Os cursos com restrição de sala (por exemplo, UC14 e UC22 → Lab01) devem utilizar obrigatoriamente essa sala.


In [140]:
from constraint import *
from collections import defaultdict

## Função split_uc_day
Esta função implementa uma hard constraint que garante que as duas aulas (D1 e D2) de cada UC ocorram em dias diferentes.
Ela adiciona restrições ao problema para que split_days(a) e split_days(b) não coincidam, assegurando a separação entre os dias das aulas.

Esta constraint foi inicialmente concebida como uma soft constraint, uma vez que representa uma preferência de organização do horário e não uma impossibilidade física.

No entanto, durante o desenvolvimento e testes do agente, verificou-se que ao manter esta restrição como soft, o solver entrava num loop de execução infinito.
Isto acontecia porque o espaço de procura (todas as combinações possíveis de horários) se tornava demasiado extenso e o solver, ao não eliminar antecipadamente as combinações inválidas, acabava por tentar enumerar um número impraticável de possibilidades.

Para resolver este problema e garantir a terminação do processo de procura, a restrição foi promovida a hard constraint.
Desta forma, o solver descarta logo no início todas as combinações em que as aulas da mesma UC ocorrem no mesmo dia, reduzindo drasticamente o espaço de procura e tornando o processo computacionalmente viável.

In [141]:
def split_uc_day(problem):
    for classe, ucs in classes.items():
        for uc in ucs:
            d1 = f"{uc}_{classe}_D1"
            d2 = f"{uc}_{classe}_D2"
            problem.addConstraint(lambda a, b: split_days(a) !=split_days(b), (d1, d2))
    return problem

## Função attribute_uc_day
Esta função implementa uma hard constraint que garante que todas as aulas (D1 e D2) de uma mesma turma ocorram em slots diferentes.
Ela adiciona uma restrição AllDifferentConstraint para impedir que duas UCs da mesma turma sejam atribuídas ao mesmo horário.


In [142]:
def attribute_uc_day(problem):              
    for classe, ucs in classes.items():
        vars_classe = []
        for uc in ucs:
            vars_classe.append(f"{uc}_{classe}_D1")
            vars_classe.append(f"{uc}_{classe}_D2")
        problem.addConstraint(AllDifferentConstraint(), tuple(vars_classe))       
    return problem

## Função split_teacher_slot
Esta função implementa uma hard constraint que garante que um mesmo professor não tenha duas aulas atribuídas no mesmo horário.
Ela agrupa os slots de cada professor e aplica uma restrição AllDifferentConstraint() para evitar sobreposição de aulas.


In [143]:
def split_teacher_slot(problem):
    #Cria dicionario para guardar os slots dos professores
    teacher_slots = {}
    for classe, ucs in classes.items():
        for uc in ucs:
            #juntar as duas UC iguais, por professor
            teacher = teacher_by_uc[uc]
            teacher_slots.setdefault(teacher, []).extend([
                f"{uc}_{classe}_D1",
                f"{uc}_{classe}_D2",
            ])
    #adiciona a restricao a cada professor
    for teacher, slots in teacher_slots.items():
        problem.addConstraint(AllDifferentConstraint(), slots)
    return problem

## Função limit_uc_day
Esta função implementa uma hard constraint que limita o número de aulas por dia para cada turma.
Ela aplica a restrição max_three_per_day sobre todas as UCs de uma turma, garantindo que cada classe tenha no máximo três aulas no mesmo dia.


In [144]:
def limit_uc_day(problem):
    for classe, ucs in classes.items():
        vars_classe = []
        for uc in ucs:
            vars_classe.append(f"{uc}_{classe}_D1")
            vars_classe.append(f"{uc}_{classe}_D2")

        problem.addConstraint(max_three_per_day, vars_classe)

    return problem

## Função attribute_room
Esta função implementa uma hard constraint que garante que duas aulas presenciais não ocorram na mesma sala e no mesmo horário.
Ela adiciona uma restrição que permite a coincidência apenas se alguma das aulas for "ONLINE", evitando conflitos de sala física.


In [145]:
def attribute_room(problem):
    same_uc = []
    for classe, ucs in classes.items():
        for uc in ucs:
            same_uc.append((f"{uc}_{classe}_R1", f"{uc}_{classe}_D1"))
            same_uc.append((f"{uc}_{classe}_R2", f"{uc}_{classe}_D2"))

    for i in range(len(same_uc)):
        for j in range(i + 1, len(same_uc)):
            rA, dA = same_uc[i]
            rB, dB = same_uc[j]

            problem.addConstraint(
                lambda ra, sa, rb, sb: not (ra != "ONLINE" and rb != "ONLINE" and ra == rb and sa == sb),
                (rA, dA, rB, dB)
            )

    return problem

## Função split_room
Esta função implementa uma hard constraint que define as salas atribuídas a cada UC, garantindo coerência entre sessões.
Ela cria variáveis de sala para cada aula (R1, R2), restringe UCs com sala obrigatória (como "lab01") e impõe que ambas as sessões da mesma UC ocorram na mesma sala.


In [146]:
def split_room(problem):
    for classe, ucs in classes.items():
        for uc in ucs:
            # cria variaveis de sala para cada sessao da UC
            r1 = f"{uc}_{classe}_R1"
            r2 = f"{uc}_{classe}_R2"

            if uc in room_schedule.get("lab01", []):
                # se a UC tem sala obrigatória, só pode usar essa sala
                obligatory_room1 = ["lab01"]
                obligatory_room2 = ["lab01"]
            else:
                # se a UC não tem sala obrigatória, pode usar qualquer sala disponível
                obligatory_room1 = available_rooms
                obligatory_room2 = available_rooms

            problem.addVariable(r1, obligatory_room1)
            problem.addVariable(r2, obligatory_room2)

            problem.addConstraint(lambda a, b: a == b, (r1, r2))
    return problem

## Função schedule_teacher_uc
Esta função implementa uma hard constraint que define os horários possíveis para cada UC de acordo com a disponibilidade do professor.
Ela cria variáveis de horário (D1, D2) para cada UC e restringe seus valores aos slots disponíveis do respectivo professor.

In [147]:
def schedule_teacher_uc(problem):
    for classe, ucs in classes.items():
        for uc in ucs:
            teacher = teacher_by_uc[uc]
            available = teacher_available_slots[teacher]
            problem.addVariable(f"{uc}_{classe}_D1", available)
            problem.addVariable(f"{uc}_{classe}_D2", available)
    return problem

# Heurísticas aplicadas para Soft Constraints

As soft constraints foram implementadas com base em heurísticas de avaliação e otimização, que permitem distinguir entre soluções válidas e soluções de maior qualidade.
Estas heurísticas não invalidam soluções, mas atribuem pontuações que refletem a sua adequação aos objetivos do problema.

As heurísticas utilizadas foram:

Heurística de compactação semanal (try_four_days_week): privilegia horários em que cada turma tem aulas distribuídas por apenas quatro dias da semana, reduzindo a fragmentação do horário.

Heurística de consecutividade (try_consecutive_ucs): favorece horários em que as aulas diárias ocorrem de forma consecutiva, evitando intervalos entre blocos e aumentando a eficiência do tempo letivo.

Heurística de distribuição temporal (try_uc_distinct_days): incentiva que as duas aulas da mesma unidade curricular ocorram em dias diferentes, melhorando a distribuição do estudo e reduzindo sobrecarga num único dia.


# Restrições Suaves (Soft Constraints)

- As aulas do mesmo curso devem ocorrer em dias distintos.
- Cada turma deve, se possível, concentrar as aulas em apenas quatro dias por semana.
- As aulas de cada dia devem ser consecutivas, evitando intervalos entre elas.


## Função try_four_days_week

Esta função implementa uma soft constraint que verifica se cada turma tem aulas distribuídas por exatamente quatro dias da semana.
Para cada turma, são identificados os dias correspondentes aos slots de cada disciplina.
Se uma turma tiver aulas em menos ou mais de quatro dias, a solução é rejeitada (None).
Caso todas as turmas cumpram o critério, a solução é considerada válida.

In [148]:
def try_four_days_week(solution):
    for class_name, ucs_dict in solution.items():
        days = set()
        for uc, info in ucs_dict.items():
            slot = info.get("slot")
            days.add(slot // 4 + 1)
        if(len(days) != 4):
            return None

    return solution

## Função try_consecutive_ucs

Esta função implementa uma soft constraint que assegura que as aulas de cada turma ocorrem em blocos consecutivos dentro do mesmo dia.
Para cada turma, as aulas são agrupadas por dia e ordenadas por número de bloco.
Se for detetado um intervalo entre blocos (diferença superior a 1), a solução é rejeitada (None).
Caso contrário, é considerada válida.

Esta restrição melhora a qualidade dos horários, evitando períodos livres entre aulas.

In [149]:
def try_consecutive_ucs(solution):
    for class_name, ucs_dict in solution.items():
        slots_por_dia = defaultdict(list)

        for uc, info in ucs_dict.items():
            slot = info.get("slot")
            dia = slot // 4
            slots_por_dia[dia].append(slot)

        for slots in slots_por_dia.values():
            slots.sort()
            if len(slots) > 1:
                for i in range(len(slots) - 1):
                    if slots[i+1] - slots[i] != 1:
                        return None

    return solution

## Função try_uc_distinct_days


Esta função implementa uma soft constraint que verifica se as duas aulas da mesma unidade curricular (por exemplo, UC12_D1 e UC12_D2) ocorrem em dias diferentes.
Para cada turma, as aulas são agrupadas por UC base (ignorando o sufixo _D1 ou _D2) e são identificados os dias correspondentes aos respetivos slots.
Se ambas as aulas de uma UC ocorrerem no mesmo dia, a solução é rejeitada (None).
Caso todas as UCs estejam distribuídas por dias distintos, a solução é considerada válida.

In [150]:
def try_uc_distinct_days(solution):
    for class_name, ucs_dict in solution.items():
        # group all UC sessions (D1, D2, ...) by UC base name
        grouped_by_uc = defaultdict(list)

        for uc_code, info in ucs_dict.items():
            slot = info.get("slot")
            dia = slot // 4
            uc_base = uc_code.split("_D")[0]  # e.g. UC12_D1 -> UC12
            grouped_by_uc[uc_base].append(dia)

        for uc, days in grouped_by_uc.items():
            if len(days) != len(set(days)):
                return None

    return solution

# Funções de Apoio

Este capítulo reúne um conjunto de funções auxiliares desenvolvidas para apoiar o funcionamento do agente e facilitar a análise dos resultados obtidos.
Estas funções não representam restrições do problema (hard ou soft constraints), mas sim mecanismos de apoio, utilizados em fases posteriores do processo, como:

- avaliação das soluções geradas (atribuição de pontuações e seleção da melhor solução),
- formatação e impressão dos horários gerados.

O objetivo deste módulo é tornar o sistema mais completo, interpretável e reutilizável, permitindo não só identificar a melhor solução, mas também apresentar e guardar os resultados de forma estruturada.

## Função score_solutions

Esta função implementa o processo de avaliação das soluções geradas, atribuindo uma pontuação a cada uma com base na consecutividade das aulas.
Para cada solução, percorre todas as turmas e agrupa as aulas por dia, ordenando os respetivos slots.
Sempre que duas aulas da mesma turma ocorrem em blocos consecutivos dentro do mesmo dia, é atribuído um ponto ao score dessa solução.
No final, a função devolve uma lista de dicionários, onde cada elemento contém a solução e a sua pontuação associada.

In [151]:
def score_solutions(solutions, solutionKey, scoreKey):
    solutions_scores = []
    for solution in solutions:
        for class_name, ucs_dict in solution.items():
            slots_por_dia = defaultdict(list)
            solution_scores_aux = {}
            score = 0

            for uc, info in ucs_dict.items():
                slot = info.get("slot")
                dia = slot // 4
                slots_por_dia[dia].append(slot)

            for slots in slots_por_dia.values():
                slots.sort()
                if len(slots) > 1:
                    for i in range(len(slots) - 1):
                        if slots[i+1] - slots[i] == 1:
                            score += 1

            solution_scores_aux[solutionKey] = solution
            solution_scores_aux[scoreKey] = score
            solutions_scores.append(solution_scores_aux)

    return solutions_scores

## Função break_solution_class

Esta função implementa uma função de apoio que transforma a solução plana gerada pelo solver.
Cada variável tem o formato UCXX_tYY_DZ ou UCXX_tYY_RZ, representando, respetivamente, o slot (bloco horário) e a sala de uma unidade curricular.
A função separa o nome da variável em partes, identifica se corresponde a um slot ou a uma room, e combina ambas as informações num único dicionário.
Desta forma, garante que UC11_t01_D1 e UC11_t01_R1 ficam ligados na mesma entrada, resultando numa estrutura final mais organizada e fácil de interpretar pelo programa.

In [152]:
def break_solution_class(solution):
    final = {}
    for var, value in solution.items():
        parts = var.split("_")
        if len(parts) != 3:
            continue

        uc, classe, suffix = parts
        final.setdefault(classe, {})

        if suffix.startswith("D"):
            label = f"{uc}_{suffix}"
            final[classe].setdefault(label, {})["slot"] = int(value)
        elif suffix.startswith("R"):
            label = f"{uc}_D{suffix[1:]}"
            final[classe].setdefault(label, {})["room"] = value

    return final

## Função max_three_per_day

Esta função é utilizada dentro de uma hard constrain (limit_uc_day) para verificar o número máximo de aulas permitidas por dia.

A função max_three_per_day recebe os slots atribuídos às aulas de uma turma, converte cada slot no respetivo dia da semana (slot // 4) e conta quantas aulas ocorrem em cada um dos cinco dias.

Ao ser usada numa hard constraint, garante que nenhum dia tem mais de três aulas, devolvendo True apenas quando esta condição é respeitada.

In [153]:
def max_three_per_day(*slots):
    days = [s // 4 for s in slots]
    return all(days.count(day) <= 3 for day in range(5))

## Função split_days

Esta função é uma função auxiliar utilizada dentro de uma hard constraint (split_uc_day).
A sua função é converter o número do slot (0–19) no dia da semana correspondente (0–4), através da divisão inteira por 4 (slot // 4), uma vez que existem 4 blocos por dia.

Desta forma, permite que a restrição split_uc_day compare os dias das duas aulas da mesma unidade curricular e garanta que estas ocorrem em dias distintos, assegurando uma distribuição semanal coerente

In [154]:
def split_days(slots):
    return slots // 4

## Função print_schedule_with_rooms_teachers

Esta função implementa uma função de apoio responsável por apresentar o horário completo gerado pelo solver, de forma legível e estruturada por turma.
A função percorre a solução final, que contém todas as variáveis resultantes da resolução do problema de agendamento. Cada turma é processada individualmente, agrupando as aulas por dia da semana e bloco horário. Para cada unidade curricular (UC), são apresentados o nome da UC, o bloco, a sala e o professor responsável.

O mapeamento entre unidades curriculares e professores é obtido a partir do dicionário `teacher_by_uc`, definido nos dados do problema. Os blocos são convertidos em dias e faixas horárias de duas horas cada, conforme o modelo temporal do problema
Desta forma, a função facilita a leitura e validação da solução final, permitindo verificar
se as restrições de salas, horários e professores foram corretamente aplicadas.

Esta função é uma função auxiliar utilizada dentro de uma hard constraint (split_uc_day).
A sua função é converter o número do slot (0–19) no dia da semana correspondente (0–4), através da divisão inteira por 4 (slot // 4), uma vez que existem 4 blocos por dia.
Desta forma, permite que a restrição split_uc_day compare os dias das duas aulas da mesma unidade curricular e garanta que estas ocorrem em dias distintos, assegurando uma distribuição semanal coerente.

In [155]:
def print_schedule_with_rooms_teachers(solution):
    days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]

    for class_name, courses in solution.items():
        print(f"\n🧩 Class {class_name}")
        print("-" * (len(class_name) + 10))
        
        entries = []
        for course, info in courses.items():
            slot = info.get("slot")
            if slot is None or not isinstance(slot, int):
                continue
            day_index = slot // 4
            block = slot % 4 + 1
            room = info.get("room", "?")
            teacher = teacher_by_uc.get(course.split("_")[0], "?")
            entries.append((day_index, block, course, room, teacher))

        # Sort by day then by block
        entries.sort()

        # Print grouped by day
        current_day = None
        for day_index, block, course, room, teacher in entries:
            if day_index != current_day:
                print(f"  {days[day_index]}:")
                current_day = day_index
            print(f"    - {course}  (Block {block}, Room {room}, Teacher {teacher})")

    print("\n✅ End of schedule")

# Execução do Agente

Este capítulo apresenta o código principal responsável por executar o agente de planeamento de horários.
É nesta fase que o agente é executado, são aplicadas todas as hard e soft constraints,
e é feita a avaliação das soluções para determinar a melhor solução possível.
Além disso, o tempo de execução total do solver é medido para permitir uma análise de desempenho.

## Descrição Geral

O processo de execução do agente segue os seguintes passos:

1. Inicialização do problema: criação de uma instância Problem() da biblioteca constraint.
2. Aplicação das hard constraints: são definidas todas as restrições obrigatórias, como a separação de professores, turmas e salas.
3. Aplicação das soft constraints: são aplicadas restrições de preferência, como aulas consecutivas, UC em dias distintos e quatro dias por semana.
4. Geração de soluções: o solver é executado com base nas restrições definidas.
5. Avaliação das soluções: cada solução é pontuada pela função score_solutions.
6. Seleção da melhor solução: é escolhida a que apresenta a pontuação mais alta.
7. Cálculo do tempo de execução: é medido o tempo total entre o início e o fim da procura.

In [156]:
from constraint import *
import time

problem = Problem()
t = schedule_teacher_uc(problem)
t = split_room(t)
t = attribute_room(t)
t = split_uc_day(t)
t = attribute_uc_day(t)
t = limit_uc_day(t)
t = split_teacher_slot(t)

final_solutions = []
feasible_solutions = 0
optimal_solutions = 0
number_of_tries = 100

four_days_week_soft = 0
uc_distinct_days_soft = 0
consecutive_ucs_soft = 0

#Get Solutions
start_time = time.time()  # medir tempo de execução
hard_solutions = []
for i, solution in enumerate(t.getSolutionIter()):
    sol = break_solution_class(solution)
    feasible_solutions += 1
    if sol != None:
        hard_solutions.append(sol)
    if i >= number_of_tries:
        break

if(len(hard_solutions) > 0):
    #four days a week (soft)
    first_soft = []
    for i in range(len(hard_solutions)):
        sol = try_four_days_week(hard_solutions[i])
        if sol != None:
            first_soft.append(sol)
            four_days_week_soft += 1

    if(len(first_soft) > 0):
        #Distinct ucs in same day (soft)
        second_soft = []
        for i in range(len(first_soft)):
            sol = try_uc_distinct_days(first_soft[i])
            if (sol != None):
                second_soft.append(sol)
                uc_distinct_days_soft += 1

        if(len(second_soft) > 0):
            #Consecutive ucs in same day (soft)
            third_soft = []
            for i in range(len(second_soft)):
                 sol = try_consecutive_ucs(second_soft[i])
                 if(sol != None):
                     third_soft.append(sol)
                     consecutive_ucs_soft += 1

            if (len(third_soft) > 0):
                final_solutions = third_soft
            else:
                final_solutions = second_soft
        else:
            final_solutions = first_soft
    else:
        final_solutions = hard_solutions

    # Gets the best solution
    solutions_score = score_solutions(final_solutions, "solution", "score")
    best_solution = None
    for i in range(len(solutions_score)):
        if best_solution == None:
            best_solution = solutions_score[i]

        if (solutions_score[i]["score"] > best_solution["score"]):
            best_solution = solutions_score[i]

    end_time = time.time()
    execution_time = end_time - start_time
    print_schedule_with_rooms_teachers(best_solution["solution"])
    print("--------------SOLUTIONS--------------")
    print("Possible Solutions: ", number_of_tries + 1)
    print("Feasible Solutions: ", feasible_solutions)
    print("Optimal Solutions: ", optimal_solutions)
    print(f"Execution Time: {execution_time:.2f} seconds")
    print("----------------DEBUG----------------")
    print("four_days_week_soft", four_days_week_soft)
    print("uc_distinct_days_soft", uc_distinct_days_soft)
    print("consecutive_ucs_soft", consecutive_ucs_soft)
else:
    print("No solution found")



🧩 Class t01
-------------
  Tuesday:
    - UC15_D2  (Block 3, Room lab01, Teacher sue)
    - UC12_D2  (Block 4, Room r05, Teacher mike)
  Wednesday:
    - UC11_D2  (Block 3, Room lab01, Teacher jo)
    - UC12_D1  (Block 4, Room r05, Teacher mike)
  Thursday:
    - UC14_D2  (Block 1, Room lab01, Teacher rob)
    - UC13_D2  (Block 2, Room lab01, Teacher rob)
    - UC15_D1  (Block 4, Room lab01, Teacher sue)
  Friday:
    - UC14_D1  (Block 1, Room lab01, Teacher rob)
    - UC13_D1  (Block 2, Room lab01, Teacher rob)
    - UC11_D1  (Block 4, Room lab01, Teacher jo)

🧩 Class t02
-------------
  Tuesday:
    - UC25_D2  (Block 2, Room r05, Teacher sue)
    - UC23_D2  (Block 3, Room r05, Teacher mike)
    - UC22_D2  (Block 4, Room lab01, Teacher jo)
  Wednesday:
    - UC22_D1  (Block 2, Room lab01, Teacher jo)
    - UC23_D1  (Block 3, Room r05, Teacher mike)
  Thursday:
    - UC25_D1  (Block 1, Room r05, Teacher sue)
    - UC24_D2  (Block 3, Room lab01, Teacher rob)
    - UC21_D2  (Block 4, R

# Conclusão

O desenvolvimento deste projeto permitiu criar um agente inteligente de geração de horários capaz de produzir, avaliar e selecionar soluções otimizadas com base num conjunto de hard e soft constraints.
Ao longo do processo, o principal objetivo foi garantir a viabilidade das soluções geradas, introduzindo simultaneamente mecanismos de avaliação que permitissem melhorar a sua qualidade de acordo com preferências realistas, como aulas consecutivas, semanas de quatro dias e aulas da mesma unidade curricular em dias distintos.

A utilização da biblioteca Python Constraint facilitou a modelação do problema de satisfação de restrições (CSP), enquanto as funções auxiliares desenvolvidas aumentaram a flexibilidade e a interpretabilidade dos resultados, permitindo também a impressão estruturada dos horários.

Em termos de resultados, o agente conseguiu gerar soluções viáveis e otimizadas, com desempenho mensurável através do tempo de execução e das pontuações atribuídas pela função de avaliação.
Apesar da complexidade combinatória inerente ao domínio dos horários, a estrutura modular da implementação permitiu analisar os resultados de forma eficiente e propor melhorias futuras, como a conversão de algumas restrições rígidas em restrições suaves, aplicando penalizações graduais em vez de exclusões diretas, tornando o sistema mais flexível.

Em suma, este projeto proporcionou uma compreensão prática sobre a resolução de problemas baseada em restrições, o equilíbrio entre restrições rígidas (hard) e preferenciais (soft), e o uso do Python como uma ferramenta poderosa na construção de agentes.